In [ ]:
from dotenv import load_dotenv

load_dotenv()

import duckdb
from src.settings import settings

In [ ]:
conn = duckdb.connect(settings.DUCKDB_DATABASE)

In [ ]:
conn.sql("""
SELECT * FROM public.cchain__climate_atmosphere
ORDER BY uuid
LIMIT 20
""").pl()